# Strategies in Federated Learning

Welcome to the next part of the federated learning tutorial. In previous parts of this tutorial, we introduced federated learning with PyTorch and Flower ([part 1](https://flower.dev/docs/tutorial/Flower-1-Intro-to-FL-PyTorch.html)).

In this notebook, we'll begin to customize the federated learning system we built in the introductory notebook (again, using [Flower](https://flower.dev/) and [PyTorch](https://pytorch.org/)).

> [Star Flower on GitHub](https://github.com/adap/flower) ⭐️ and join the Flower community on Slack to connect, ask questions, and get help: [Join Slack](https://flower.dev/join-slack) 🌼 We'd love to hear from you in the `#introductions` channel! And if anything is unclear, head over to the `#questions` channel.

Let's move beyond FedAvg with Flower Strategies!

## Preparation

Before we begin with the actual code, let's make sure that we have everything we need.

### Installing dependencies

First, we install the necessary packages:

In [ ]:
!pip install -q flwr[simulation] torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 KB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 8.4 

Now that we have all dependencies installed, we can import everything we need for this tutorial:

In [ ]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

import flwr as fl

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cpu using PyTorch 2.0.0+cu118 and Flower 1.3.0


It is possible to switch to a runtime that has GPU acceleration enabled (on Google Colab: `Runtime > Change runtime type > Hardware acclerator: GPU > Save`). Note, however, that Google Colab is not always able to offer GPU acceleration. If you see an error related to GPU availability in one of the following sections, consider switching back to CPU-based execution by setting `DEVICE = torch.device("cpu")`. If the runtime has GPU acceleration enabled, you should see the output `Training on cuda`, otherwise it'll say `Training on cpu`.

### Data loading

Let's now load the CIFAR-10 training and test set, partition them into ten smaller datasets (each split into training and validation set), and wrap everything in their own `DataLoader`. We introduce a new parameter `num_clients` which allows us to call `load_datasets` with different numbers of clients.

In [ ]:
NUM_CLIENTS = 10


def load_datasets(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader


trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

100%|██████████| 170498071/170498071 [00:03<00:00, 48709162.30it/s]


Extracting ./dataset/cifar-10-python.tar.gz to ./dataset
Files already downloaded and verified


### Model training/evaluation

Let's continue with the usual model definition (including `set_parameters` and `get_parameters`), training and test functions:

In [ ]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

### Flower client

To implement the Flower client, we (again) create a subclass of `flwr.client.NumPyClient` and implement the three methods `get_parameters`, `fit`, and `evaluate`. Here, we also pass the `cid` to the client and use it log additional details:

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

## Strategy customization

So far, everything should look familiar if you've worked through the introductory notebook. With that, we're ready to introduce a number of new features. 

### Server-side parameter **initialization**

Flower, by default, initializes the global model by asking one random client for the initial parameters. In many cases, we want more control over parameter initialization though. Flower therefore allows you to directly pass the initial parameters to the Strategy:

In [ ]:
# Create an instance of the model and get the parameters
params = get_parameters(Net())

# Pass parameters to the Strategy for server-side parameter initialization
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(params),
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-04-08 00:41:19,760 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-04-08 00:41:21,157	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-04-08 00:41:22,579 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 7925617460.0, 'object_store_memory': 3962808729.0, 'CPU': 2.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'memory': 7925617460.0, 'object_store_memory': 3962808729.0, 'CPU': 2.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0}
INFO flwr 2023-04-08 00:41:22,588 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-04-08 00:41:22,595 | server.py:266 | Using initial parameters provided by strategy
INFO:flwr:Using initial parameters provided by strategy
INFO flwr 2023

(launch_and_fit pid=1099) [Client 2] fit, config: {}
(launch_and_fit pid=1098) [Client 6] fit, config: {}
(launch_and_fit pid=1098) Epoch 1: train loss 0.06596717238426208, accuracy 0.21288888888888888
(launch_and_fit pid=1099) Epoch 1: train loss 0.06593053787946701, accuracy 0.222
(launch_and_fit pid=1098) [Client 0] fit, config: {}


DEBUG flwr 2023-04-08 00:41:37,273 | server.py:229 | fit_round 1 received 3 results and 0 failures
DEBUG:flwr:fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-04-08 00:41:37,287 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-04-08 00:41:37,289 | server.py:165 | evaluate_round 1: strategy sampled 3 clients (out of 10)
DEBUG:flwr:evaluate_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=1098) Epoch 1: train loss 0.06495313346385956, accuracy 0.23355555555555554
(launch_and_evaluate pid=1099) [Client 6] evaluate, config: {}
(launch_and_evaluate pid=1098) [Client 5] evaluate, config: {}


DEBUG flwr 2023-04-08 00:41:43,273 | server.py:179 | evaluate_round 1 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 1 received 3 results and 0 failures
WARNING flwr 2023-04-08 00:41:43,280 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-04-08 00:41:43,282 | server.py:215 | fit_round 2: strategy sampled 3 clients (out of 10)
DEBUG:flwr:fit_round 2: strategy sampled 3 clients (out of 10)


...

(launch_and_fit pid=1099) Epoch 1: train loss 0.05417599156498909, accuracy 0.3671111111111111
(launch_and_evaluate pid=1099) [Client 4] evaluate, config: {}
(launch_and_evaluate pid=1098) [Client 8] evaluate, config: {}


DEBUG flwr 2023-04-08 00:42:14,977 | server.py:179 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-04-08 00:42:14,983 | server.py:144 | FL finished in 52.371506576
INFO:flwr:FL finished in 52.371506576
INFO flwr 2023-04-08 00:42:14,988 | app.py:202 | app_fit: losses_distributed [(1, 0.0656187702814738), (2, 0.05534022736549377), (3, 0.05250716082255045)]
INFO:flwr:app_fit: losses_distributed [(1, 0.0656187702814738), (2, 0.05534022736549377), (3, 0.05250716082255045)]


(launch_and_evaluate pid=1098) [Client 2] evaluate, config: {}


INFO flwr 2023-04-08 00:42:14,992 | app.py:203 | app_fit: metrics_distributed {}
INFO:flwr:app_fit: metrics_distributed {}
INFO flwr 2023-04-08 00:42:14,995 | app.py:204 | app_fit: losses_centralized []
INFO:flwr:app_fit: losses_centralized []
INFO flwr 2023-04-08 00:42:14,997 | app.py:205 | app_fit: metrics_centralized {}
INFO:flwr:app_fit: metrics_centralized {}


History (loss, distributed):
	round 1: 0.0656187702814738
	round 2: 0.05534022736549377
	round 3: 0.05250716082255045

Passing `initial_parameters` to the `FedAvg` strategy prevents Flower from asking one of the clients for the initial parameters. If we look closely, we can see that the logs do not show any calls to the `FlowerClient.get_parameters` method.

### Starting with a customized strategy

We've seen the function `start_simulation` before. It accepts a number of arguments, amongst them the `client_fn` used to create `FlowerClient` instances, the number of clients to simulate `num_clients`, the number of rounds `num_rounds`, and the strategy.

The strategy encapsulates the federated learning approach/algorithm, for example, `FedAvg` or `FedAdagrad`. Let's try to use a different strategy this time:

In [ ]:
# Create FedAdam strategy
strategy = fl.server.strategy.FedAdagrad(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-04-08 00:42:20,611 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-04-08 00:42:25,479	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-04-08 00:42:27,611 | app.py:179 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7914221568.0, 'node:172.28.0.12': 1.0, 'GPU': 1.0, 'object_store_memory': 3957110784.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7914221568.0, 'node:172.28.0.12': 1.0, 'GPU': 1.0, 'object_store_memory': 3957110784.0}
INFO flwr 2023-04-08 00:42:27,615 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-04-08 00:42:27,624 | server.py:266 | Using initial parameters provided by strategy
INFO:flwr:Using initial parameters provided by strategy
INFO flwr 2023

(launch_and_fit pid=1696) [Client 6] fit, config: {}
(launch_and_fit pid=1697) [Client 4] fit, config: {}
(launch_and_fit pid=1696) Epoch 1: train loss 0.066340871155262, accuracy 0.2117777777777778
(launch_and_fit pid=1697) Epoch 1: train loss 0.06558381766080856, accuracy 0.22133333333333333
(launch_and_fit pid=1697) [Client 5] fit, config: {}


DEBUG flwr 2023-04-08 00:42:41,486 | server.py:229 | fit_round 1 received 3 results and 0 failures
DEBUG:flwr:fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-04-08 00:42:41,505 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-04-08 00:42:41,515 | server.py:165 | evaluate_round 1: strategy sampled 3 clients (out of 10)
DEBUG:flwr:evaluate_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=1697) Epoch 1: train loss 0.06482434272766113, accuracy 0.2328888888888889
(launch_and_evaluate pid=1696) [Client 9] evaluate, config: {}
(launch_and_evaluate pid=1697) [Client 3] evaluate, config: {}


DEBUG flwr 2023-04-08 00:42:46,267 | server.py:179 | evaluate_round 1 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 1 received 3 results and 0 failures
WARNING flwr 2023-04-08 00:42:46,269 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-04-08 00:42:46,276 | server.py:215 | fit_round 2: strategy sampled 3 clients (out of 10)
DEBUG:flwr:fit_round 2: strategy sampled 3 clients (out of 10)


...

DEBUG flwr 2023-04-08 00:43:13,255 | server.py:229 | fit_round 3 received 3 results and 0 failures
DEBUG:flwr:fit_round 3 received 3 results and 0 failures
DEBUG flwr 2023-04-08 00:43:13,284 | server.py:165 | evaluate_round 3: strategy sampled 3 clients (out of 10)
DEBUG:flwr:evaluate_round 3: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=1697) Epoch 1: train loss 0.09872926026582718, accuracy 0.15244444444444444
(launch_and_evaluate pid=1696) [Client 0] evaluate, config: {}
(launch_and_evaluate pid=1697) [Client 6] evaluate, config: {}


DEBUG flwr 2023-04-08 00:43:17,483 | server.py:179 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-04-08 00:43:17,486 | server.py:144 | FL finished in 49.849861616
INFO:flwr:FL finished in 49.849861616
INFO flwr 2023-04-08 00:43:17,488 | app.py:202 | app_fit: losses_distributed [(1, 6.707209197998047), (2, 0.5413721510569255), (3, 0.08104761473337808)]
INFO:flwr:app_fit: losses_distributed [(1, 6.707209197998047), (2, 0.5413721510569255), (3, 0.08104761473337808)]
INFO flwr 2023-04-08 00:43:17,490 | app.py:203 | app_fit: metrics_distributed {}
INFO:flwr:app_fit: metrics_distributed {}
INFO flwr 2023-04-08 00:43:17,492 | app.py:204 | app_fit: losses_centralized []
INFO:flwr:app_fit: losses_centralized []
INFO flwr 2023-04-08 00:43:17,494 | app.py:205 | app_fit: metrics_centralized {}
INFO:flwr:app_fit: metrics_centralized {}


(launch_and_evaluate pid=1697) [Client 9] evaluate, config: {}


History (loss, distributed):
	round 1: 6.707209197998047
	round 2: 0.5413721510569255
	round 3: 0.08104761473337808

## Server-side parameter **evaluation**

Flower can evaluate the aggregated model on the server-side or on the client-side. Client-side and server-side evaluation are similar in some ways, but different in others.

**Centralized Evaluation** (or *server-side evaluation*) is conceptually simple: it works the same way that evaluation in centralized machine learning does. If there is a server-side dataset that can be used for evaluation purposes, then that's great. We can evaluate the newly aggregated model after each round of training without having to send the model to clients. We're also fortunate in the sense that our entire evaluation dataset is available at all times.

**Federated Evaluation** (or *client-side evaluation*) is more complex, but also more powerful: it doesn't require a centralized dataset and allows us to evaluate models over a larger set of data, which often yields more realistic evaluation results. In fact, many scenarios require us to use **Federated Evaluation** if we want to get representative evaluation results at all. But this power comes at a cost: once we start to evaluate on the client side, we should be aware that our evaluation dataset can change over consecutive rounds of learning if those clients are not always available. Moreover, the dataset held by each client can also change over consecutive rounds. This can lead to evaluation results that are not stable, so even if we would not change the model, we'd see our evaluation results fluctuate over consecutive rounds.

We've seen how federated evaluation works on the client side (i.e., by implementing the `evaluate` method in `FlowerClient`). Now let's see how we can evaluate aggregated model parameters on the server-side:

In [ ]:
# The `evaluate` function will be by Flower called after every round
def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = Net().to(DEVICE)
    valloader = valloaders[0]
    set_parameters(net, parameters)  # Update model with the latest parameters
    loss, accuracy = test(net, valloader)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

In [ ]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    evaluate_fn=evaluate,  # Pass the evaluation function
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-04-08 00:43:41,419 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-04-08 00:43:47,028	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-04-08 00:43:48,073 | app.py:179 | Flower VCE: Ray initialized with resources: {'object_store_memory': 3956847820.0, 'node:172.28.0.12': 1.0, 'GPU': 1.0, 'memory': 7913695643.0, 'CPU': 2.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'object_store_memory': 3956847820.0, 'node:172.28.0.12': 1.0, 'GPU': 1.0, 'memory': 7913695643.0, 'CPU': 2.0}
INFO flwr 2023-04-08 00:43:48,084 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-04-08 00:43:48,089 | server.py:266 | Using initial parameters provided by strategy
INFO:flwr:Using initial parameters provided by strategy
INFO flwr 2023

Server-side evaluation loss 0.07364050245285034 / accuracy 0.092
(launch_and_fit pid=2381) [Client 2] fit, config: {}
(launch_and_fit pid=2380) [Client 5] fit, config: {}
(launch_and_fit pid=2381) Epoch 1: train loss 0.06500381976366043, accuracy 0.22022222222222224
(launch_and_fit pid=2381) [Client 1] fit, config: {}
(launch_and_fit pid=2380) Epoch 1: train loss 0.06474390625953674, accuracy 0.2242222222222222


DEBUG flwr 2023-04-08 00:44:03,419 | server.py:229 | fit_round 1 received 3 results and 0 failures
DEBUG:flwr:fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-04-08 00:44:03,431 | fedavg.py:242 | No fit_metrics_aggregation_fn provided


(launch_and_fit pid=2381) Epoch 1: train loss 0.06434320658445358, accuracy 0.23955555555555555


INFO flwr 2023-04-08 00:44:03,620 | server.py:116 | fit progress: (1, 0.06291225051879883, {'accuracy': 0.282}, 15.182561483000029)
INFO:flwr:fit progress: (1, 0.06291225051879883, {'accuracy': 0.282}, 15.182561483000029)
DEBUG flwr 2023-04-08 00:44:03,626 | server.py:165 | evaluate_round 1: strategy sampled 3 clients (out of 10)
DEBUG:flwr:evaluate_round 1: strategy sampled 3 clients (out of 10)


...

INFO flwr 2023-04-08 00:44:36,649 | server.py:116 | fit progress: (3, 0.05262840723991394, {'accuracy': 0.398}, 48.210805311)
INFO:flwr:fit progress: (3, 0.05262840723991394, {'accuracy': 0.398}, 48.210805311)
DEBUG flwr 2023-04-08 00:44:36,657 | server.py:165 | evaluate_round 3: strategy sampled 3 clients (out of 10)
DEBUG:flwr:evaluate_round 3: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.05262840723991394 / accuracy 0.398
(launch_and_evaluate pid=2381) [Client 6] evaluate, config: {}
(launch_and_evaluate pid=2380) [Client 0] evaluate, config: {}


DEBUG flwr 2023-04-08 00:44:42,434 | server.py:179 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-04-08 00:44:42,437 | server.py:144 | FL finished in 53.998930855000026
INFO:flwr:FL finished in 53.998930855000026
INFO flwr 2023-04-08 00:44:42,439 | app.py:202 | app_fit: losses_distributed [(1, 0.06243557929992676), (2, 0.05606243872642517), (3, 0.05256911897659302)]
INFO:flwr:app_fit: losses_distributed [(1, 0.06243557929992676), (2, 0.05606243872642517), (3, 0.05256911897659302)]
INFO flwr 2023-04-08 00:44:42,440 | app.py:203 | app_fit: metrics_distributed {}
INFO:flwr:app_fit: metrics_distributed {}
INFO flwr 2023-04-08 00:44:42,442 | app.py:204 | app_fit: losses_centralized [(0, 0.07364050245285034), (1, 0.06291225051879883), (2, 0.056906822681427), (3, 0.05262840723991394)]
INFO:flwr:app_fit: losses_centralized [(0, 0.07364050245285034), (1, 0.06291225051879883), (2, 0.056906822681427), (3, 0.0526284

(launch_and_evaluate pid=2381) [Client 9] evaluate, config: {}


History (loss, distributed):
	round 1: 0.06243557929992676
	round 2: 0.05606243872642517
	round 3: 0.05256911897659302
History (loss, centralized):
	round 0: 0.07364050245285034
	round 1: 0.06291225051879883
	round 2: 0.056906822681427
	round 3: 0.05262840723991394
History (metrics, centralized):
{'accuracy': [(0, 0.092), (1, 0.282), (2, 0.364), (3, 0.398)]}

## Sending/receiving arbitrary values to/from clients

In some situations, we want to configure client-side execution (trainig, evaluation) from the server-side. One example for that is the server asking the clients to train for a certain number of local epochs. Flower provides a way to send configuration values from the server to the clients using a dictionary. Let's look at an example where the clients receive values from the server through the `config` parameter in `fit` (`config` is also available in `evaluate`). The `fit` method receives the configuration dictionary through the `config` parameter and can then read values from this dictionary. In this example, it reads `server_round` and `local_epochs` and uses those values to improve the logging and configure the number of local training epochs:

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        # Read values from config
        server_round = config["server_round"]
        local_epochs = config["local_epochs"]

        # Use values provided by the config
        print(f"[Client {self.cid}, round {server_round}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

So how can we  send this config dictionary from server to clients? The built-in Flower Strategies provide way to do this, and it works similarly to the way server-side evaluation works. We provide a function to the strategy, and the strategy calls this function for every round of federated learning:

In [ ]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        "local_epochs": 1 if server_round < 2 else 2,  #
    }
    return config

Next, we'll just pass this function to the FedAvg strategy before starting the simulation:

In [ ]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    evaluate_fn=evaluate,
    on_fit_config_fn=fit_config,  # Pass the fit_config function
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-04-08 00:44:53,093 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-04-08 00:44:57,471	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-04-08 00:44:58,850 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 7912978023.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3956489011.0, 'CPU': 2.0, 'GPU': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'memory': 7912978023.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3956489011.0, 'CPU': 2.0, 'GPU': 1.0}
INFO flwr 2023-04-08 00:44:58,856 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-04-08 00:44:58,861 | server.py:266 | Using initial parameters provided by strategy
INFO:flwr:Using initial parameters provided by strategy
INFO flwr 2023

Server-side evaluation loss 0.07373963689804078 / accuracy 0.096
(launch_and_fit pid=2998) [Client 7, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(launch_and_fit pid=2999) [Client 3, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(launch_and_fit pid=2998) Epoch 1: train loss 0.06477630883455276, accuracy 0.23022222222222222
(launch_and_fit pid=2999) Epoch 1: train loss 0.0647486224770546, accuracy 0.21688888888888888
(launch_and_fit pid=2998) [Client 5, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}


DEBUG flwr 2023-04-08 00:45:14,718 | server.py:229 | fit_round 1 received 3 results and 0 failures
DEBUG:flwr:fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-04-08 00:45:14,741 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
INFO flwr 2023-04-08 00:45:14,948 | server.py:116 | fit progress: (1, 0.06205886769294739, {'accuracy': 0.282}, 15.73631741700001)
INFO:flwr:fit progress: (1, 0.06205886769294739, {'accuracy': 0.282}, 15.73631741700001)
DEBUG flwr 2023-04-08 00:45:14,954 | server.py:165 | evaluate_round 1: strategy sampled 3 clients (out of 10)
DEBUG:flwr:evaluate_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=2998) Epoch 1: train loss 0.06472079455852509, accuracy 0.22577777777777777
Server-side evaluation loss 0.06205886769294739 / accuracy 0.282
(launch_and_evaluate pid=2998) [Client 1] evaluate, config: {}
(launch_and_evaluate pid=2999) [Client 9] evaluate, config: {}


DEBUG flwr 2023-04-08 00:45:18,919 | server.py:179 | evaluate_round 1 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 1 received 3 results and 0 failures
WARNING flwr 2023-04-08 00:45:18,926 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-04-08 00:45:18,928 | server.py:215 | fit_round 2: strategy sampled 3 clients (out of 10)
DEBUG:flwr:fit_round 2: strategy sampled 3 clients (out of 10)


...

DEBUG flwr 2023-04-08 00:46:03,167 | server.py:229 | fit_round 3 received 3 results and 0 failures
DEBUG:flwr:fit_round 3 received 3 results and 0 failures
INFO flwr 2023-04-08 00:46:03,358 | server.py:116 | fit progress: (3, 0.04997988176345825, {'accuracy': 0.424}, 64.14670738900003)
INFO:flwr:fit progress: (3, 0.04997988176345825, {'accuracy': 0.424}, 64.14670738900003)
DEBUG flwr 2023-04-08 00:46:03,361 | server.py:165 | evaluate_round 3: strategy sampled 3 clients (out of 10)
DEBUG:flwr:evaluate_round 3: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=2998) Epoch 2: train loss 0.04647305980324745, accuracy 0.45444444444444443
Server-side evaluation loss 0.04997988176345825 / accuracy 0.424
(launch_and_evaluate pid=2998) [Client 1] evaluate, config: {}
(launch_and_evaluate pid=2999) [Client 4] evaluate, config: {}


DEBUG flwr 2023-04-08 00:46:07,269 | server.py:179 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-04-08 00:46:07,274 | server.py:144 | FL finished in 68.06253614500002
INFO:flwr:FL finished in 68.06253614500002
INFO flwr 2023-04-08 00:46:07,279 | app.py:202 | app_fit: losses_distributed [(1, 0.06195564826329549), (2, 0.051947560787200926), (3, 0.04767045005162557)]
INFO:flwr:app_fit: losses_distributed [(1, 0.06195564826329549), (2, 0.051947560787200926), (3, 0.04767045005162557)]
INFO flwr 2023-04-08 00:46:07,282 | app.py:203 | app_fit: metrics_distributed {}
INFO:flwr:app_fit: metrics_distributed {}
INFO flwr 2023-04-08 00:46:07,292 | app.py:204 | app_fit: losses_centralized [(0, 0.07373963689804078), (1, 0.06205886769294739), (2, 0.051531314849853516), (3, 0.04997988176345825)]
INFO:flwr:app_fit: losses_centralized [(0, 0.07373963689804078), (1, 0.06205886769294739), (2, 0.051531314849853516), (3, 0.0

(launch_and_evaluate pid=2999) [Client 8] evaluate, config: {}


History (loss, distributed):
	round 1: 0.06195564826329549
	round 2: 0.051947560787200926
	round 3: 0.04767045005162557
History (loss, centralized):
	round 0: 0.07373963689804078
	round 1: 0.06205886769294739
	round 2: 0.051531314849853516
	round 3: 0.04997988176345825
History (metrics, centralized):
{'accuracy': [(0, 0.096), (1, 0.282), (2, 0.4), (3, 0.424)]}

As we can see, the client logs now include the current round of federated learning (which they read from the `config` dictionary). We can also configure local training to run for one epoch during the first and second round of federated learning, and then for two epochs during the third round.

Clients can also return arbitrary values to the server. To do so, they return a dictionary from `fit` and/or `evaluate`. We have seen and used this concept throughout this notebook without mentioning it explicitly: our `FlowerClient` returns a dictionary containing a custom key/value pair as the third return value in `evaluate`.

## Scaling federated learning

As a last step in this notebook, let's see how we can use Flower to experiment with a large number of clients.

In [ ]:
NUM_CLIENTS = 1000

trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


We now have 1000 partitions, each holding 45 training and 5 validation examples. Given that the number of training examples on each client is quite small, we should probably train the model a bit longer, so we configure the clients to perform 3 local training epochs. We should also adjust the fraction of clients selected for training during each round (we don't want all 1000 clients participating in every round), so we adjust `fraction_fit` to `0.05`, which means that only 5% of available clients (so 50 clients) will be selected for training each round:


In [ ]:
def fit_config(server_round: int):
    config = {
        "server_round": server_round,
        "local_epochs": 3,
    }
    return config


strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.025,  # Train on 25 clients (each round)
    fraction_evaluate=0.05,  # Evaluate on 50 clients (each round)
    min_fit_clients=20,
    min_evaluate_clients=40,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    on_fit_config_fn=fit_config,
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-04-08 00:46:24,842 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-04-08 00:46:28,948	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-04-08 00:46:30,399 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 7913592423.0, 'CPU': 2.0, 'node:172.28.0.12': 1.0, 'GPU': 1.0, 'object_store_memory': 3956796211.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'memory': 7913592423.0, 'CPU': 2.0, 'node:172.28.0.12': 1.0, 'GPU': 1.0, 'object_store_memory': 3956796211.0}
INFO flwr 2023-04-08 00:46:30,411 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-04-08 00:46:30,420 | server.py:266 | Using initial parameters provided by strategy
INFO:flwr:Using initial parameters provided by strategy
INFO flwr 2023

(launch_and_fit pid=3708) [Client 712, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=3707) [Client 325, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=3707) Epoch 1: train loss 0.1021730825304985, accuracy 0.08888888888888889
(launch_and_fit pid=3708) Epoch 1: train loss 0.10190457105636597, accuracy 0.08888888888888889
(launch_and_fit pid=3708) Epoch 2: train loss 0.10196076333522797, accuracy 0.1111111111111111
(launch_and_fit pid=3707) Epoch 2: train loss 0.10204503685235977, accuracy 0.08888888888888889
(launch_and_fit pid=3707) Epoch 3: train loss 0.10117954760789871, accuracy 0.17777777777777778
(launch_and_fit pid=3708) Epoch 3: train loss 0.10096663236618042, accuracy 0.24444444444444444
(launch_and_fit pid=3708) [Client 394, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=3707) [Client 728, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=3708) 

DEBUG flwr 2023-04-08 00:47:17,152 | server.py:229 | fit_round 1 received 25 results and 0 failures
DEBUG:flwr:fit_round 1 received 25 results and 0 failures
WARNING flwr 2023-04-08 00:47:17,320 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-04-08 00:47:17,328 | server.py:165 | evaluate_round 1: strategy sampled 50 clients (out of 1000)
DEBUG:flwr:evaluate_round 1: strategy sampled 50 clients (out of 1000)


(launch_and_fit pid=3708) Epoch 2: train loss 0.10218459367752075, accuracy 0.08888888888888889
(launch_and_fit pid=3708) Epoch 3: train loss 0.10193277895450592, accuracy 0.17777777777777778


(raylet) Spilled 3687 MiB, 48 objects, write throughput 229 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.


...

DEBUG flwr 2023-04-08 00:51:32,093 | server.py:229 | fit_round 3 received 25 results and 0 failures
DEBUG:flwr:fit_round 3 received 25 results and 0 failures
DEBUG flwr 2023-04-08 00:51:32,218 | server.py:165 | evaluate_round 3: strategy sampled 50 clients (out of 1000)
DEBUG:flwr:evaluate_round 3: strategy sampled 50 clients (out of 1000)


(launch_and_fit pid=3707) Epoch 2: train loss 0.09834984689950943, accuracy 0.17777777777777778
(launch_and_fit pid=3708) Epoch 2: train loss 0.1005382314324379, accuracy 0.1111111111111111
(launch_and_fit pid=3708) Epoch 3: train loss 0.09806687384843826, accuracy 0.2
(launch_and_fit pid=3707) Epoch 3: train loss 0.09823138266801834, accuracy 0.2
(launch_and_evaluate pid=3708) [Client 402] evaluate, config: {}
(launch_and_evaluate pid=3707) [Client 394] evaluate, config: {}
(launch_and_evaluate pid=3708) [Client 82] evaluate, config: {}
(launch_and_evaluate pid=3707) [Client 555] evaluate, config: {}
(launch_and_evaluate pid=3708) [Client 189] evaluate, config: {}
(launch_and_evaluate pid=3707) [Client 939] evaluate, config: {}
(launch_and_evaluate pid=3707) [Client 993] evaluate, config: {}
(launch_and_evaluate pid=3708) [Client 230] evaluate, config: {}
(launch_and_evaluate pid=3708) [Client 459] evaluate, config: {}
(launch_and_evaluate pid=3707) [Client 921] evaluate, config: {}
(

DEBUG flwr 2023-04-08 00:52:58,657 | server.py:179 | evaluate_round 3 received 50 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 50 results and 0 failures
INFO flwr 2023-04-08 00:52:58,663 | server.py:144 | FL finished in 388.21898469399997
INFO:flwr:FL finished in 388.21898469399997
INFO flwr 2023-04-08 00:52:58,667 | app.py:202 | app_fit: losses_distributed [(1, 0.45937115573883064), (2, 0.45756772041320787), (3, 0.45411158275604246)]
INFO:flwr:app_fit: losses_distributed [(1, 0.45937115573883064), (2, 0.45756772041320787), (3, 0.45411158275604246)]
INFO flwr 2023-04-08 00:52:58,674 | app.py:203 | app_fit: metrics_distributed {}
INFO:flwr:app_fit: metrics_distributed {}
INFO flwr 2023-04-08 00:52:58,676 | app.py:204 | app_fit: losses_centralized []
INFO:flwr:app_fit: losses_centralized []
INFO flwr 2023-04-08 00:52:58,678 | app.py:205 | app_fit: metrics_centralized {}
INFO:flwr:app_fit: metrics_centralized {}


(launch_and_evaluate pid=3708) [Client 436] evaluate, config: {}
(launch_and_evaluate pid=3707) [Client 279] evaluate, config: {}


History (loss, distributed):
	round 1: 0.45937115573883064
	round 2: 0.45756772041320787
	round 3: 0.45411158275604246

## Recap

In this notebook, we've seen how we can gradually enhance our system by customizing the strategy, initializing parameters on the server side, choosing a different strategy, and evaluating models on the server-side. That's quite a bit of flexibility with so little code, right?

In the later sections, we've seen how we can communicate arbitrary values between server and clients to fully customize client-side execution. With that capability, we built a large-scale Federated Learning simulation using the Flower Virtual Client Engine and ran an experiment involving 1000 clients in the same workload - all in a Jupyter Notebook!

## Next steps

Before you continue, make sure to join the Flower community on Slack: [Join Slack](https://flower.dev/join-slack/)

There's a dedicated `#questions` channel if you need help, but we'd also love to hear who you are in `#introductions`!

The [Flower Federated Learning Tutorial - Part 3 [WIP]](https://flower.dev/docs/tutorial/Flower-3-Building-a-Strategy-PyTorch.html) shows how to build a fully custom `Strategy` from scratch.